In [23]:
# Function to replace non-numeric values with '-'
def replace_non_numeric(x):
    try:
        # Try converting to numeric; if it fails, replace with '-'
        pd.to_numeric(x)
        return x
    except (ValueError, TypeError):
        return '-'



In [25]:
##Cleaning the data 
import pandas as pd

# Read the original CSV file
input_file = "11.csv"
df = pd.read_csv(input_file)
df['company'] = df['company'].replace({
    'Amazon.com': 'Amazon',
    'Amazon.com, Inc.': 'Amazon',
    'Alphabet Inc':'Alphabet',
    'Amazon.com Inc.': 'Amazon.com',
    'The Goldman Sachs Group, Inc.': 'Goldman Sachs',
    'Goldman Sachs Group, Inc.':'Goldman Sachs',
    'Tesla, Inc.':'Tesla',
    'Tesla Motors, Inc.': 'Tesla',
    'Tesla Motors':'Tesla',
    'Valero Energy Corporation': 'Valero Energy',
    'Cardinal Health':'Cardinal Health, Inc.',
    'BERKSHIRE HATHAWAY INC.': 'Berkshire Hathaway',
    'Berkshire Hathaway Inc' : 'Berkshire Hathaway',
    '"Cardinal Health, Inc."':'Cardinal Health',
    'Chevron Corporation' : 'Chevron'

})
# Apply the function to the desired column
df['capex'] = df['capex'].apply(replace_non_numeric)
df['company'] = df['company'].drop(columns=['Company Name'])
# Save the rearranged DataFrame to a new CSV file
output_file = "cleaned_financial_analysis_results11.csv"
df.to_csv(output_file, index=False)

print(f"cleaned data saved to {output_file}.")

cleaned data saved to cleaned_financial_analysis_results11.csv.


In [35]:
## Rearranged input capex with quarter_year with company name 
import pandas as pd

# Read the original CSV file
input_file = "cleaned_financial_analysis_results11.csv"
df = pd.read_csv(input_file)
df['quarter'] = df['quarter'].astype(str)
df['year'] = df['year'].astype(str)

# Add 'Q' prefix to 'quarter'
df['quarter'] = 'Q' + df['quarter']
# Convert 'quarter' and 'year' columns into a single 'Quarter-Year' column with an underscore
df['Quarter-Year'] = df['quarter'] + "_" + df['year'].astype(str)

# Pivot the DataFrame
pivot_df = df.pivot_table(index='company', columns='Quarter-Year', values='capex', aggfunc='first')

# Reset the index to make 'company' a column instead of an index
pivot_df.reset_index(inplace=True)

# Define a function to sort columns in the desired order
def sort_columns(df):
    # Extract the current columns
    columns = df.columns.tolist()
    
    # Extract the company column and the Quarter-Year columns
    company_col = columns[0]
    quarter_cols = columns[1:]
    
    # Generate sorted columns list: Start with the most recent quarters
    sorted_quarters = sorted(quarter_cols, key=lambda x: (x.split('_')[1], x.split('_')[0]), reverse=True)
    
    # Combine sorted columns with the company column
    sorted_columns = [company_col] + sorted_quarters
    return sorted_columns

# Reorder the columns
sorted_columns = sort_columns(pivot_df)
pivot_df = pivot_df[sorted_columns]
pivot_df['company'].unique()
pivot_df['company'] = pivot_df['company'].str.replace('"', ' ', regex=False)
# # Save the rearranged DataFrame to a new CSV file
output_file = "rearranged_financial_analysis_results11.csv"
pivot_df.to_csv(output_file, index=False)

print(f"Rearranged data saved to {output_file}.")

Rearranged data saved to rearranged_financial_analysis_results11.csv.


In [37]:
import pandas as pd

# Read the CSV file
input_file = "rearranged_financial_analysis_results11.csv"
df = pd.read_csv(input_file)

# Define a function to compare capex values between consecutive columns
def compare_columns(row):
    comparisons = []
    for i in range(1, len(row) - 1):  # Skip the 'company' column
        current_value = row[i]
        next_value = row[i + 1]
        
        if pd.isna(current_value) or pd.isna(next_value):
            comparisons.append("Data not available")
        else:
            try:
                current_value = float(current_value)
                next_value = float(next_value)
                if next_value < current_value:
                    comparisons.append("increase")
                elif next_value > current_value:
                    comparisons.append("decrease")
                else:
                    comparisons.append("unchanged")
            except ValueError:
                comparisons.append("Data not available")
    
    return comparisons

# Apply the comparison function to each row
comparison_results = df.apply(lambda row: compare_columns(row), axis=1)

# Replace the original values in the DataFrame with the comparison results
for i, col in enumerate(df.columns[1:-1]):  # Exclude 'company' and the last quarter
    df[col] = comparison_results.apply(lambda x: x[i])

# Save the comparison results to a new CSV file
output_file = "capex_comparison_results12.csv"
df.to_csv(output_file, index=False)

print(f"Capex comparison results saved to {output_file}.")

Capex comparison results saved to capex_comparison_results12.csv.


/tmp/ipykernel_5103/2170017367.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_value = row[i]
/tmp/ipykernel_5103/2170017367.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i + 1]
